In [1]:
import time
import statistics as stats
import multiprocessing
import time 
import random
import time
from tqdm import tqdm
from datetime import timedelta
from datetime import datetime
start_program = time.time()
from multiprocessing import *
import pandas as pd
import numpy as np
from threading import Thread
import time
import subprocess 


In [2]:
results = [[] for i in range(5)]

n_it = 100


n_st = number_stations = 10
n_s = number_sensors = 100

def_st = 1
def_s = 3
def_r = 'day'


set_date = [random.random() for i in range(500)]


In [3]:

def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))



def random_date(start, end, prop, dform = '%Y-%m-%dT%H:%M:%S'):
    return str_time_prop(start, end, dform, prop)
    
    
def to_pm(v):
    return str(round(v[0][0],2)) + "$" + '\\' + "pm$" + str(round(v[1][0],2))


# Queries

In [4]:
# A simple class
# attribute

c_q1 = """select time, id_station, <sid> FROM <db> where id_station in <stid>
AND time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
AND time < TIMESTAMP '<timestamp>';"""

# c_q1 = """select time, s<sid> FROM d1_wide where id_station='st<stid>'
# AND time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
# AND time < TIMESTAMP '<timestamp>';"""

c_q2 = """select time, id_station, <sid> FROM <db> where id_station in <stid>
AND time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
AND time < TIMESTAMP '<timestamp>' and <sfilter>""" # s<sid> > 0.95;

c_q3 = """SELECT id_station, <avg_s> FROM <db> 
WHERE time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
AND time < TIMESTAMP '<timestamp>' 
AND id_station in <stid>
GROUP BY id_station;"""   #avg(s<sid>)

c_q4 = """SELECT id_station, EXTRACT(YEAR FROM time) AS "year",
date_trunc('month', time) AS "month", 
date_trunc('day', time) AS "day", 
date_trunc('hour', time) AS "hour", 
<avg_s> 
FROM <db> where  time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
AND time < TIMESTAMP '<timestamp>' 
AND id_station in <stid>
GROUP BY id_station, "year", "month", "day", "hour";""" # AVG(s<sid>) AS avg_s<sid>

c_q5 = """select time, id_station, <sid> FROM <db> where id_station in <stid>
	AND time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
	AND time < TIMESTAMP '<timestamp>' ORDER BY
	id_station ASC,
	time ASC WITH FILL STEP 5
	INTERPOLATE ( <sid>, id_station AS id_station );
"""



In [5]:
from clickhouse_driver import Client
from clickhouse_driver import connect as connect_ClickHouse
from tqdm import tqdm 
import time

class ClickHouse:
 
    # A sample method 
    @staticmethod
    def query(query, db, max_d, rangesUnit, n_it, start_date = "2019-04-01T00:00:00", stop_date = "2019-04-30T00:00:00", n_st = 1, n_s = 10):
#         client = Client('diufrm102')
        conn = connect_ClickHouse("clickhouse://diufrm102")
        cursor = conn.cursor()
#         print(client.execute('SHOW TABLES'))
#         return None, None
        duration = max_d
        results = [[],[]]       
        runtimes = []
        full_time = time.time()
        for i in tqdm(range(n_it)):
#             time.sleep(0.5)
            date = random_date(start_date, stop_date, set_date[(duration*i)%500], dform = '%Y-%m-%dT%H:%M:%S')
            temp = query.replace("<timestamp>", date)
            temp = temp.replace("<nb>", str(duration))
            temp = temp.replace("<rangesUnit>", str(rangesUnit))
#             temp = temp.replace("<sid>", str(set_s[(duration*i)%500]))
            # st
            li = ['st' + str(z) for z in random.sample(range(number_stations), n_st)]
            q = '(' + "'" + li[0] + "'"
            for j in li[1:]:
                q += ', ' + "'" + j + "'"
            q += ")"
            temp = temp.replace("<stid>", q)
            
            # sensor
            li = ['s' + str(z) for z in random.sample(range(number_sensors), n_s)]
            q = li[0]
            q_filter = '(' + li[0] + ' > 0.95'
            q_avg = 'avg(' + li[0] + ')'
            for j in li[1:]:
                q += ', ' + j
#                 q_filter += ' OR ' + j + ' > 0.95'
                q_avg += ', ' + 'avg(' + j + ')'
            temp = temp.replace("<db>", db)
            temp = temp.replace("<sid>", q)
            temp = temp.replace("<sfilter>", q_filter + ')')
            temp = temp.replace("<avg_s>", q_avg)
            

            start = time.time()
#             print(temp)
            cursor.execute(temp)
            cursor.fetchall()
#            print(len(cursor.fetchall()))
            diff = (time.time()-start)*1000
#                 print(temp, diff)
            runtimes.append(diff)
            if time.time() - full_time > 20 and i > 5: 
                break              
#             print(temp)
        results[0].append(stats.mean(runtimes))
#             results[1].append(percentile(runtimes,85))
        results[1].append(stats.stdev(runtimes))
        conn.close()
        return results[0], results[1]
    
    @staticmethod
    def insert_file(batch_size,rate, st):
#         template_sql = "INSERT INTO d1_wide_slow_ingest (time, id_station"
#         for i in range(100):
#             template_sql += ", s" + str(i)
#         template_sql += ") VALUES ('%s', '%s',"  + ','.join(100* [' %s']) + ');'

        conn = connect_ClickHouse("clickhouse://diufrm102")
        cur = conn.cursor()
        file = '/home/abdel/venv2/bin/insertion-100/short_ts' + str(st) + '.csv'
        f = open(file, "r")
        f = f.read()
        f = f.splitlines()[1:]
        f = ["\n".join(f[i:i+rate])  for i in range(0, len(f), rate)]
        start = time.time()
        a = 0
        print('insertion st' + str(st) + ' started')
        while not stop_insertion[st]: 
            starttime = time.time()
            cur.execute("INSERT INTO d1_wide_slow_ingest FORMAT CSV " + f[a])
            a += 1
            diff = 1.0 - ((time.time() - starttime))
            try: 
                time.sleep(diff)
            except: 
                print(diff)
                
    @staticmethod
    def insert_file_old(batch_size,rate, st):
#         template_sql = "INSERT INTO d1_wide_slow_ingest (time, id_station"
#         for i in range(100):
#             template_sql += ", s" + str(i)
#         template_sql += ") VALUES ('%s', '%s',"  + ','.join(100* [' %s']) + ');'

        conn = connect_ClickHouse("clickhouse://diufrm102")
        cur = conn.cursor()
        a = 1
        file = '/home/abdel/venv2/bin/insertion-100/short_ts' + str(st) + '.csv'
        start = time.time()
        print('insertion st' + str(st) + ' started')
        while not stop_insertion[st]: 
            starttime = time.time()
            cmd = """sed '1,""" + str(a) + """d;""" + str(a + rate + 1) + """,$d' """ + file
            cur.execute("INSERT INTO d1_wide_slow_ingest FORMAT CSV " + subprocess.getoutput(cmd))
            a += rate
            diff = 1.0 - ((time.time() - starttime))
            try: 
                time.sleep(diff)
            except: 
                print(diff)                

    @staticmethod
    def insert(batch_size, rate, st):
        template_sql = "INSERT INTO d1_wide_slow_ingest (time, id_station"
        for i in range(100):
            template_sql += ", s" + str(i)
        template_sql += ") VALUES"

        conn = connect_ClickHouse("clickhouse://diufrm102")
        cur = conn.cursor()

        date_time = datetime.strptime('2019-04-30T00:00:00', '%Y-%m-%dT%H:%M:%S') 
        delta = timedelta(seconds=10)
        synthetic = [[date_time + k * delta
                  , 'st' + str(st)
                ] + [random.random() for i in range(100)] for k in range(rate*500)]
        print('insertion st' + str(st) + ' started')
#         print('synthetic generated, inserting... ')
        a = 0
        start = time.time()
        while not stop_insertion[st]: 
            starttime = time.time()

            for _ in range(batch_size, max(rate, batch_size) + 1, batch_size):
                cur.executemany(template_sql,  synthetic[a : a + rate])
                a += rate
                date_time += delta * rate

            diff = 1.0 - ((time.time() - starttime))
            try: 
                time.sleep(diff)
            except: 
                print(diff)

    @staticmethod
    def insert_by_point(rate, st):
        template_sql = "INSERT INTO d1_wide_slow_ingest (time, id_station"
        for i in range(100):
            template_sql += ", s" + str(i)
        template_sql += ") VALUES ('%s', '%s',"  + ','.join(100* [' %s']) + ');'

        conn = connect_ClickHouse("clickhouse://diufrm102")
        cur = conn.cursor()
        date_time = datetime.strptime('2019-04-30T00:00:00', '%Y-%m-%dT%H:%M:%S') 
        delta = timedelta(seconds=10)
        synthetic = [[(delta * k + date_time).strftime('%Y-%m-%dT%H:%M:%S')
                          , 'st' + str(st)] + [random.random() for i in range(100)] for k in range(rate * 500)]
        print('insertion ' + str(st) + ' started')
#         print('synthetic generated, inserting... ')
        a = 0
        start = time.time()
        while not stop_insertion[st]: 
            starttime = time.time()

            for e in range(a, a + rate):
#                 print(synthetic[e])
#                 date_time += delta
                sql = template_sql % tuple(synthetic[e % len(synthetic)])
                cur.execute(sql)
#                 conn.commit()
            a += rate
            diff = 1.0 - ((time.time() - starttime))
            try: 
                time.sleep(diff)
            except: 
                print(diff)

                
        
    @staticmethod
    def run_all(start, stop, db):
        results[0].append(to_pm(ClickHouse.query(c_q1, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[1].append(to_pm(ClickHouse.query(c_q2, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[2].append(to_pm(ClickHouse.query(c_q3, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[3].append(to_pm(ClickHouse.query(c_q4, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[4].append(to_pm(ClickHouse.query(c_q5, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
    
    
    
    

In [6]:
ClickHouse.run_all("2019-04-01T00:00:00", "2019-04-30T00:00:00", "d1_wide")

100%|█████████████████████████████████| 100/100 [00:08<00:00, 12.45it/s]


In [ ]:
n_threads = 100
stop_insertion = [False for i in range(n_threads)]
rate = 20

# 20K dp/s

# Launch insertion
for i in range(n_threads):  
    t1 = Thread(target = ClickHouse.insert_file, args=(100_000, rate, i))
#     t1 = Thread(target = ClickHouse.insert_by_point, args=(rate, i))
    t1.setDaemon(True)
    t1.start()

# insert_new(200, 1)

In [ ]:
time.sleep(3 * 8640 // rate)

In [7]:
ClickHouse.run_all("2019-04-01T00:00:00", "2019-04-30T00:00:00", "d1_wide_slow_ingest")

100%|█████████████████████████████████| 100/100 [00:08<00:00, 11.20it/s]


In [8]:
ClickHouse.run_all("2019-05-01T01:00:00", "2019-05-02T00:00:00", "d1_wide_slow_ingest")

100%|█████████████████████████████████| 100/100 [00:07<00:00, 12.70it/s]


In [9]:
ClickHouse.run_all("2019-04-29T01:00:00", "2019-05-02T00:00:00", "d1_wide_slow_ingest")

100%|█████████████████████████████████| 100/100 [00:08<00:00, 11.87it/s]


In [ ]:
stop_insertion = [True for i in range(n_threads)]


In [10]:
results 

[['31.01$\\pm$2.4', '35.2$\\pm$6.15', '34.42$\\pm$4.06', '34.6$\\pm$5.64'],
 ['10.86$\\pm$1.39', '13.95$\\pm$4.81', '14.75$\\pm$4.46', '14.84$\\pm$4.51'],
 ['10.91$\\pm$1.17', '14.18$\\pm$3.73', '14.28$\\pm$3.1', '13.8$\\pm$3.87'],
 ['13.94$\\pm$1.5', '17.92$\\pm$5.17', '17.66$\\pm$4.73', '17.22$\\pm$5.27'],
 ['79.81$\\pm$5.8', '88.83$\\pm$11.4', '78.22$\\pm$7.6', '83.71$\\pm$10.01']]

In [ ]:
runtimes = [[float(results[i][j].split('$')[0]) for j in range(len(results[i]))][:4] for i in range(5)]
variance = [[float(results[i][j].split('$')[2]) for j in range(len(results[i]))][:4] for i in range(5)]

In [ ]:
runtimes = list(map(list, zip(*runtimes)))
variance = list(map(list, zip(*variance)))


In [ ]:
runtimes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
X = np.arange(5)
labels = ['query'+str(1+i) for i in range(5)]
fig = plt.figure(figsize=(8, 6))
ax = fig.add_axes([0,0,1,1])
for i in range(4):
    ax.bar(X +0.15*i, runtimes[i], yerr = variance[i], width = 0.15)
ax.legend(['offline', 'online old', 'online new', 'online mixed', ])
ax.set_xticklabels(labels)
ax.set_xticks(np.arange(len(labels)))
ax.yaxis.grid(True)
plt.title('ClickHouse offline vs. online, rate: '+ str(rate) + ' x ' + str(n_threads))
plt.tight_layout()

# ax.bar(X+0.25, data[1], color = 'g', width = 0.25)
# ax.bar(X+0.5, data[2], color = 'r', width = 0.25)

In [ ]:
# conn = connect_ClickHouse("clickhouse://diufrm102")
# cur = conn.cursor()
# cur.execute("ALTER TABLE d1_wide_slow_ingest DELETE where time > TIMESTAMP '2019-04-30T00:00:00';")